<a href="https://colab.research.google.com/github/hungviet0304/DS_Final_project/blob/master/DS_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Crawling data**
 - Data source: https://www.imdb.com/*italicized text*
 - In 8 categories: action, comedy, Adventure,Animation, Crime, Romance, Documentary, Biography
 - Colecting 10000 samples each category

In [0]:
import lxml
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [0]:
def crawling(url_next_page, genre):

  position = 1
  url = url_next_page.format(genre, position).lower()
  print(url)

  page = get(url)
  s = BeautifulSoup(page.content,'lxml')

  #page title
  page_title = s.find("h1", class_ = "header").text.replace("\n","")
  body = s.find(id = 'main')
  content = body.find_all("div", class_ = "lister-item mode-advanced")
  total_results = 0
  # movie_title = [] #
  # movie_date = []
  # movie_runtime = []#
  # movie_genre = []#
  # movie_rating = []
  # movie_score = []
  # movie_description = []
  # movie_directors = []
  # movie_stars = []
  # movie_votes = []
  # movie_gros = []
  df = pd.DataFrame(columns = ["title", "runtime","des","date","rating","genre"])
  total_results = body.find("div",class_ = 'desc')
  total_results = total_results.find('span').text.replace(",","")
  total_results = int(re.findall(r'\d+',total_results)[-1])
  print("total results: ".format(total_results))
  total_results = int(total_results)
  num_of_pages = total_results // 50 + 1 if total_results / 50 > total_results // 50 else total_results //50
  print(num_of_pages)
  num_of_pages /= 2
  count = 0
  while True:

    for movie in content:
      tmp = movie.find("h3",class_ = "lister-item-header")
      movie_title=tmp.find("a").text

      try:
        movie_runtime = movie.find('span',class_ = 'runtime').text[:-4] # only number
      except:
        movie_runtine = 0
        # print("error at runtime: " + movie_title)

      try:
        movie_genre=movie.find('span',class_= 'genre').text.replace("\n","")
      except:
        movie_genre = ""
        # print("error at genre" + movie_title)
      try:
        movie_description = movie.find_all('p',class_ = "text-muted")[-1].text.replace("\n","")
      except:
        # print("error at description: " + movie_title)
        movie_description = ""
      try:
        movie_date = tmp.find_all("span")[-1].text
        movie_date = int(re.findall(r'\d+',movie_date)[-1])
      except:
        movie_date = 0
        # print("error at date:" + movie_title)
      try:
        movie_rating = movie.find("strong").text
      except:
        movie_rating = '0'
        # print("error at rating: " + movie_title)

      df.loc[len(df)] = [movie_title, movie_runtime, movie_description,movie_date, movie_rating, movie_genre]
      if df.shape[0] >= 10000:
        # print()
        return df[:10000]
    position += 50
    url = url_next_page.format(genre,position).lower()
    page = get(url)
    s = BeautifulSoup(page.content,'lxml')

    #page title
    page_title = s.find("h1", class_ = "header").text.replace("\n","")
    body = s.find(id = 'main')
    content = body.find_all("div", class_ = "lister-item mode-advanced")
    count += 1
    # print(str(count) + " / " + str(num_of_pages))
    if count == num_of_pages:
      break
  return df

In [0]:
def crawling_genre(url, genre):
  for g in genre:
    print("crawling " + g)
    df_tmp = crawling(url,g)
    print("crawed {}".format(df_tmp.shape[0]))
    df_tmp.to_csv(g+".csv",index = False)


In [9]:
url = "https://www.imdb.com/search/title/?genres={}&languages=en&start={}&ref_=adv_nxt"
df = crawling_genre(url,['action','comedy','Adventure','Animation','Crime','Romance','Documentary','Biography'])

crawling action
https://www.imdb.com/search/title/?genres=action&languages=en&start=1&ref_=adv_nxt
total results: 
1691
crawed 10000
crawling comedy
https://www.imdb.com/search/title/?genres=comedy&languages=en&start=1&ref_=adv_nxt
total results: 
7299
crawed 10000
crawling Adventure
https://www.imdb.com/search/title/?genres=adventure&languages=en&start=1&ref_=adv_nxt
total results: 
1348
crawed 10000
crawling Animation
https://www.imdb.com/search/title/?genres=animation&languages=en&start=1&ref_=adv_nxt
total results: 
1143
crawed 10000
crawling Crime
https://www.imdb.com/search/title/?genres=crime&languages=en&start=1&ref_=adv_nxt
total results: 
1854
crawed 10000
crawling Romance
https://www.imdb.com/search/title/?genres=romance&languages=en&start=1&ref_=adv_nxt
total results: 
2826
crawed 10000
crawling Documentary
https://www.imdb.com/search/title/?genres=documentary&languages=en&start=1&ref_=adv_nxt
total results: 
3963
crawed 10000
crawling Biography
https://www.imdb.com/search/